### Fase 3 Sul Americana - Semifinais

In [76]:
import cartolafc
import pandas as pd
from difflib import get_close_matches
import json
import os

pd.set_option('display.max_columns', 50)            # permite a visualização de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os números aparecerem com duas casas decimais

# Cria uma instância da API
api = cartolafc.Api(attempts=5)

### Carregar o arquivo Excel com os classificados e Gerar um Dicionário

In [77]:
CAMINHO_ARQUIVO = "2_classificados_quartas_sula.xlsx"

# Inicialização segura
nomes_por_id = {}
dados_torneio_quartas = []

if not os.path.exists(CAMINHO_ARQUIVO):
    print("📁 Arquivo ainda não existe — aguardando fim da fase anterior.")
else:
    try:
        # Carrega o arquivo
        df_classificados = pd.read_excel(CAMINHO_ARQUIVO)

        if df_classificados.empty:
            print("📄 Arquivo existe, mas ainda está vazio. Rodadas da fase anterior não concluídas.")
        else:
            # Filtrar apenas classificados válidos
            df_validos = df_classificados[df_classificados["classificado_id"].notnull()].reset_index(drop=True)

            if len(df_validos) < 4:
                print(f"⚠️ Apenas {len(df_validos)} classificados encontrados. Aguardando rodada restante.")
            else:
                # Gerar dicionário nome → ID
                nomes_por_id = dict(zip(df_validos["classificado_id"], df_validos["classificado_nome"]))
                display(nomes_por_id)
                print("\n" + "-" * 100 + "\n")

                # Confrontos das quartas com ordem 
                dados_torneio_quartas = [
                    ("Jogo 1 (JG1)", df_validos.loc[0, "classificado_id"]),  # V1
                    ("Jogo 1 (JG1)", df_validos.loc[1, "classificado_id"]),  # V2
                    ("Jogo 2 (JG2)", df_validos.loc[2, "classificado_id"]),  # V3
                    ("Jogo 2 (JG2)", df_validos.loc[3, "classificado_id"]),  # V4
                    # ("Jogo 3 (JG3)", df_validos.loc[4, "classificado_id"]),  # V5
                    # ("Jogo 3 (JG3)", df_validos.loc[5, "classificado_id"]),  # V6
                    # ("Jogo 4 (JG4)", df_validos.loc[6, "classificado_id"]),  # V7
                    # ("Jogo 4 (JG4)", df_validos.loc[7, "classificado_id"]),  # V8                    
                ]

                print("✅ Confrontos das Quartas de Final:")
                for linha in dados_torneio_quartas:
                    print(linha)

    except Exception as e:
        print(f"❌ Erro ao processar o arquivo: {e}")


{13951133: 'KING LEONN',
 24468241: 'Grêmio imortal 37',
 25751748: 'MauHumor F.C.',
 44810918: 'lsauer fc'}


----------------------------------------------------------------------------------------------------

✅ Confrontos das Quartas de Final:
('Jogo 1 (JG1)', 13951133)
('Jogo 1 (JG1)', 24468241)
('Jogo 2 (JG2)', 25751748)
('Jogo 2 (JG2)', 44810918)


In [79]:
# Criar DataFrame base
dados_torneio_quartas = pd.DataFrame(dados_torneio_quartas, columns=["Jogo", "ID do Time"])

# Adicionar Nome do Time usando o dicionário
dados_torneio_quartas["Nome do Time"] = dados_torneio_quartas["ID do Time"].map(nomes_por_id)

# Adicionar ID no Grupo
dados_torneio_quartas["ID no Grupo"] = dados_torneio_quartas.groupby("Jogo").cumcount() + 1
dados_torneio_quartas["ID no Grupo"] = dados_torneio_quartas["ID no Grupo"].astype(str) + "_" + dados_torneio_quartas["Jogo"].str[-2]

# Reorganizar colunas
df_torneio_quartas_sula = dados_torneio_quartas[["Jogo", "ID do Time", "Nome do Time", "ID no Grupo"]]

df_sula_jogo_1 = df_torneio_quartas_sula[df_torneio_quartas_sula["Jogo"] == "Jogo 1 (JG1)"]
df_sula_jogo_2 = df_torneio_quartas_sula[df_torneio_quartas_sula["Jogo"] == "Jogo 2 (JG2)"]
# df_sula_jogo_3 = df_torneio_quartas_sula[df_torneio_quartas_sula["Jogo"] == "Jogo 3 (JG3)"]
# df_sula_jogo_4 = df_torneio_quartas_sula[df_torneio_quartas_sula["Jogo"] == "Jogo 4 (JG4)"]


# Lista de grupos
grupos = {
    "Jogo 1 (JG1)": df_sula_jogo_1,
    "Jogo 2 (JG2)": df_sula_jogo_2,
    # "Jogo 3 (JG3)": df_sula_jogo_3,
    # "Jogo 4 (JG4)": df_sula_jogo_4
}

display(df_sula_jogo_1, df_sula_jogo_2) #, df_sula_jogo_3, df_sula_jogo_4)

,Jogo,ID do Time,Nome do Time,ID no Grupo
0,Jogo 1 (JG1),13951133,KING LEONN,1_1
1,Jogo 1 (JG1),24468241,Grêmio imortal 37,2_1


,Jogo,ID do Time,Nome do Time,ID no Grupo
2,Jogo 2 (JG2),25751748,MauHumor F.C.,1_2
3,Jogo 2 (JG2),44810918,lsauer fc,2_2


### Definição dos Confrontos das 2 Rodadas da Fase 2 da Sul Americana (Rodada 15 A 16)

In [80]:
# Rodada 1 - Fase 4 Libertadores (Equivalente a 15º Rodada do Campeonato Brasileiro) 
confrontos_1a_rodada = [
    # Jogo 1 (JG1)
    ("Jogo 1 (JG1)", "1_1", "2_1"),    

    # Jogo 2 (JG2)
    ("Jogo 2 (JG2)", "1_2", "2_2"),   

    # # Jogo 3 (JG3)
    # ("Jogo 3 (JG3)", "1_3", "2_3"),

    # # Jogo 4 (JG4)
    # ("Jogo 4 (JG4)", "1_4", "2_4")
]

# Rodada 2 - Fase 4 Libertadores (Equivalente a 16º Rodada do Campeonato Brasileiro)
confrontos_2a_rodada = [
    # Jogo 1 (JG1)
    ("Jogo 1 (JG1)", "2_1", "1_1"),
    
    # Jogo 2 (JG2)
    ("Jogo 2 (JG2)", "2_2", "1_2"),

    # # Jogo 3 (JG3)
    # ("Jogo 3 (JG3)", "2_3", "1_3"),

    # # Jogo 4 (JG4)
    # ("Jogo 4 (JG4)", "2_4", "1_4") 
]


In [81]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Grupo", "Mandante_ID", "Visitante_ID"])

# Junta com df_torneio para buscar dados dos mandantes
df_mandantes = df_torneio_quartas_sula.rename(columns={
    "ID no Grupo": "Mandante_ID",
    "Nome do Time": "Mandante_Nome",
    "ID do Time": "Mandante_ID_Time"
})[["Jogo", "Mandante_ID", "Mandante_Nome", "Mandante_ID_Time"]]

# Junta com df_torneio para buscar dados dos visitantes
df_visitantes = df_torneio_quartas_sula.rename(columns={
    "ID no Grupo": "Visitante_ID",
    "Nome do Time": "Visitante_Nome",
    "ID do Time": "Visitante_ID_Time"    
})[["Jogo", "Visitante_ID", "Visitante_Nome", "Visitante_ID_Time"]]

display(df_confrontos)

,Grupo,Mandante_ID,Visitante_ID
0,Jogo 1 (JG1),1_1,2_1
1,Jogo 2 (JG2),1_2,2_2


In [82]:
# Transformar em DataFrame
df_confrontos = pd.DataFrame(confrontos_1a_rodada, columns=["Jogo", "Mandante_ID", "Visitante_ID"])
df_confrontos["Rodada"] = 17
df_rodada_15 = df_confrontos.merge(df_mandantes, on=["Jogo", "Mandante_ID"])
df_rodada_15 = df_rodada_15.merge(df_visitantes, on=["Jogo", "Visitante_ID"])

# Transformar em DataFrame
df_confrontos_2 = pd.DataFrame(confrontos_2a_rodada, columns=["Jogo", "Mandante_ID", "Visitante_ID"])
df_confrontos_2["Rodada"] = 18
df_rodada_16 = df_confrontos_2.merge(df_mandantes, on=["Jogo", "Mandante_ID"])
df_rodada_16 = df_rodada_16.merge(df_visitantes, on=["Jogo", "Visitante_ID"])

display(df_rodada_15, df_rodada_16)

,Jogo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Visitante_Nome,Visitante_ID_Time
0,Jogo 1 (JG1),1_1,2_1,17,KING LEONN,13951133,Grêmio imortal 37,24468241
1,Jogo 2 (JG2),1_2,2_2,17,MauHumor F.C.,25751748,lsauer fc,44810918


,Jogo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Visitante_Nome,Visitante_ID_Time
0,Jogo 1 (JG1),2_1,1_1,18,Grêmio imortal 37,24468241,KING LEONN,13951133
1,Jogo 2 (JG2),2_2,1_2,18,lsauer fc,44810918,MauHumor F.C.,25751748


In [83]:
df_rodadas = pd.concat([
    df_rodada_15,
    df_rodada_16
], ignore_index=True)

# Ajustar a numeração da rodada para refletir as rodadas do Cartola (Rodada 15 até 16)
df_rodadas["Rodada"] = df_rodadas["Rodada"]

df_rodadas.to_excel("3_confrontos_semi_sula.xlsx", index=False)

# Exibir os confrontos da fase 4 (Quartas de Final)
display(df_rodadas.head(8)) 

,Jogo,Mandante_ID,Visitante_ID,Rodada,Mandante_Nome,Mandante_ID_Time,Visitante_Nome,Visitante_ID_Time
0,Jogo 1 (JG1),1_1,2_1,17,KING LEONN,13951133,Grêmio imortal 37,24468241
1,Jogo 2 (JG2),1_2,2_2,17,MauHumor F.C.,25751748,lsauer fc,44810918
2,Jogo 1 (JG1),2_1,1_1,18,Grêmio imortal 37,24468241,KING LEONN,13951133
3,Jogo 2 (JG2),2_2,1_2,18,lsauer fc,44810918,MauHumor F.C.,25751748


In [84]:
# Criar lista de dicionários no formato desejado
confrontos_js_quartas = []

for _, row in df_rodadas.iterrows():
    confronto = {
        "jogo": row["Jogo"],
        "rodada": int(row["Rodada"]),
        "mandante": {
            "id": int(row["Mandante_ID_Time"]),
            "nome": row["Mandante_Nome"]
        },
        "visitante": {
            "id": int(row["Visitante_ID_Time"]),
            "nome": row["Visitante_Nome"]        }
    }
    confrontos_js_quartas.append(confronto)

# Converter para JSON formatado
json_str = json.dumps(confrontos_js_quartas, indent=2, ensure_ascii=False)

# Salvar como arquivo JS com uma variável global
with open("3_confrontos_semi_sula.js", "w", encoding="utf-8") as f:
    f.write("const confrontos_semi_sula = ")
    f.write(json_str)
    f.write(";")

In [85]:
def exibir_confrontos(df_rodadas, rodada=None, jogo=None):

    colunas = ["Rodada", "Jogo", "Mandante_Nome", "Visitante_Nome"]
    df_filtrado = df_rodadas.copy()

    df_filtrado["Rodada"] = df_filtrado["Rodada"].astype(str) + "ª Rodada"    

    if rodada is not None:
        df_filtrado = df_filtrado[df_filtrado["Rodada"] == rodada]

    if jogo is not None:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    return df_filtrado[colunas].sort_values(by=["Jogo", "Rodada"])

In [86]:
jogo = 1

# Exibir todos os confrontos do Jogo 2
display(exibir_confrontos(df_rodadas, jogo= f"Jogo {jogo} (JG{jogo})").head(8))

,Rodada,Jogo,Mandante_Nome,Visitante_Nome
0,17ª Rodada,Jogo 1 (JG1),KING LEONN,Grêmio imortal 37
2,18ª Rodada,Jogo 1 (JG1),Grêmio imortal 37,KING LEONN


In [87]:
def campeonato_comecou(api, ids_times):
    """Verifica se o campeonato já começou observando a pontuação na 1ª rodada."""
    for time_id in ids_times.values():
        try:
            pontuacao = api.time(time_id=time_id, rodada=17).ultima_pontuacao
            if pontuacao is not None:
                return True
        except cartolafc.errors.CartolaFCError:
            continue
    return False

def obter_pontuacao_por_rodada(api, time_id, rodada_atual):
    """Obtém a pontuação do time em cada rodada até a rodada atual."""
    pontuacoes = {}
    for rodada in range(17, rodada_atual):
        try:
            time_rodada = api.time(time_id=time_id, rodada=rodada)
            pontuacoes[rodada] = time_rodada.ultima_pontuacao
        except cartolafc.errors.CartolaFCError as e:
            print(f"Erro ao acessar pontuação da rodada {rodada} para o time {time_id}: {e}")
            pontuacoes[rodada] = None
    return pontuacoes



def gerar_df_pontuacoes(api, ids_times, rodada_inicio=17):
    rodada_atual = api.mercado().rodada_atual

    if not campeonato_comecou(api, ids_times):
        print("📌 A fase ainda não começou. Criando estrutura com placeholders.")
        df = pd.DataFrame(index=ids_times.keys(), columns=[f'Rodada {i}' for i in range(rodada_inicio, rodada_inicio + 2)])
        df[:] = None  # ou 0 se preferir
        return df

    # Campeonato começou
    df = pd.DataFrame()
    for nome, time_id in ids_times.items():
        pontuacoes = obter_pontuacao_por_rodada(api, time_id, rodada_atual)
        df[nome] = pd.Series(pontuacoes)

    df = df.transpose()

    # Ajustar nome das colunas para "Rodada X"
    colunas_numeradas = df.columns.tolist()
    df.columns = [f'Rodada {i}' for i in colunas_numeradas]

    # Adicionar a linha com o time líder por rodada (opcional)
    if not df.empty:
        df.loc['Lider_Rodada'] = df.idxmax()

    return df


In [88]:
ids_times = {v: k for k, v in nomes_por_id.items()}
df_pontuacoes = gerar_df_pontuacoes(api, ids_times)

if df_pontuacoes.empty:
    print("⚠️ Sem pontuações disponíveis ainda.")
else:
    display(df_pontuacoes.T)

,KING LEONN,Grêmio imortal 37,MauHumor F.C.,lsauer fc,Lider_Rodada
Rodada 17,53.09,80.10,59.81,44.16,Grêmio imortal 37
Rodada 18,92.37,73.20,105.50,82.58,MauHumor F.C.
Rodada 19,64.30,62.95,66.70,52.00,MauHumor F.C.


In [89]:
def classificacao_por_grupo(df_rodadas, df_pontuacoes):

    df_pontuacoes_times = df_pontuacoes.drop(index='Lider_Rodada', errors='ignore')
    estatisticas = {}

    for _, confronto in df_rodadas.iterrows():
        rodada = confronto["Rodada"]
        mandante = confronto["Mandante_Nome"]
        visitante = confronto["Visitante_Nome"]
        jogo = confronto["Jogo"]
        coluna_rodada = f"Rodada {rodada}"

        if mandante not in df_pontuacoes_times.index or visitante not in df_pontuacoes_times.index:
            continue
        if coluna_rodada not in df_pontuacoes_times.columns:
            continue

        pontos_mandante = df_pontuacoes_times.at[mandante, coluna_rodada]
        pontos_visitante = df_pontuacoes_times.at[visitante, coluna_rodada]

        # Ignorar confrontos ainda não disputados (com pontuação 0 ou ausente)
        if (
            pd.isnull(pontos_mandante) or pd.isnull(pontos_visitante) or
            (pontos_mandante == 0 and pontos_visitante == 0)
        ):
            continue

        for time in [mandante, visitante]:
            if jogo not in estatisticas:
                estatisticas[jogo] = {}
            if time not in estatisticas[jogo]:
                estatisticas[jogo][time] = {
                    "Pontos": 0, "Vitórias": 0, "Empates": 0, "Derrotas": 0,
                    "Total_Cartola": 0,
                    "Cartola_Sofrido": 0
                }

        # Atualizar estatísticas do jogo
        estatisticas[jogo][mandante]["Total_Cartola"] += pontos_mandante
        estatisticas[jogo][mandante]["Cartola_Sofrido"] += pontos_visitante

        estatisticas[jogo][visitante]["Total_Cartola"] += pontos_visitante
        estatisticas[jogo][visitante]["Cartola_Sofrido"] += pontos_mandante

        if pontos_mandante > pontos_visitante:
            estatisticas[jogo][mandante]["Pontos"] += 3
            estatisticas[jogo][mandante]["Vitórias"] += 1
            estatisticas[jogo][visitante]["Derrotas"] += 1
        elif pontos_mandante < pontos_visitante:
            estatisticas[jogo][visitante]["Pontos"] += 3
            estatisticas[jogo][visitante]["Vitórias"] += 1
            estatisticas[jogo][mandante]["Derrotas"] += 1
        else:
            estatisticas[jogo][mandante]["Pontos"] += 1
            estatisticas[jogo][visitante]["Pontos"] += 1
            estatisticas[jogo][mandante]["Empates"] += 1
            estatisticas[jogo][visitante]["Empates"] += 1

    # Gerar DataFrame final
    df_resultado = pd.concat([
        pd.DataFrame({
            "Jogo": jogo,
            "Nome do Time": list(times.keys()),
            "Pontos": [stats["Pontos"] for stats in times.values()],
            "Vitórias": [stats["Vitórias"] for stats in times.values()],
            "Empates": [stats["Empates"] for stats in times.values()],
            "Derrotas": [stats["Derrotas"] for stats in times.values()],
            "Total Cartola": [stats["Total_Cartola"] for stats in times.values()],
            "Cartola Sofrido": [stats["Cartola_Sofrido"] for stats in times.values()],
            "Saldo Cartola": [
                stats["Total_Cartola"] - stats["Cartola_Sofrido"] for stats in times.values()
            ]
        }) for jogo, times in estatisticas.items()
    ], ignore_index=True)

    # Ordenar e adicionar posição
    df_resultado = df_resultado.sort_values(
        by=["Jogo", "Pontos", "Vitórias", "Total Cartola", "Saldo Cartola"],
        ascending=[True, False, False, False, False]
    )

    df_resultado["Posição"] = df_resultado.groupby("Jogo").cumcount() + 1

    df_resultado_por_jogo = {
        jogo: df_resultado[df_resultado["Jogo"] == jogo] for jogo in df_resultado["Jogo"].unique()
    }

    return df_resultado, df_resultado_por_jogo


In [90]:
# Padroniza e exibe as pontuações se existirem
try:
    if 'df_pontuacoes' not in globals() or df_pontuacoes.empty:
        print("📌 Nenhuma pontuação disponível — provavelmente ainda não há classificados definidos.")
    else:
        # Padroniza os nomes no df_rodadas
        df_rodadas["Mandante_Nome"] = df_rodadas["Mandante_Nome"].str.strip()
        df_rodadas["Visitante_Nome"] = df_rodadas["Visitante_Nome"].str.strip()

        # Padroniza os índices do df_pontuacoes
        df_pontuacoes.index = df_pontuacoes.index.str.strip()

        # Exibe
        display(df_pontuacoes)

except NameError as e:
    print("❌ Variável df_rodadas ou df_pontuacoes não está definida ainda.")
except Exception as e:
    print(f"❌ Erro inesperado: {e}")


,Rodada 17,Rodada 18,Rodada 19
KING LEONN,53.09,92.37,64.30
Grêmio imortal 37,80.10,73.20,62.95
MauHumor F.C.,59.81,105.50,66.70
lsauer fc,44.16,82.58,52.00
Lider_Rodada,Grêmio imortal 37,MauHumor F.C.,MauHumor F.C.


In [91]:
try:
    # Verifica se as variáveis existem e estão válidas
    if (
        'df_rodadas' not in globals() or df_rodadas.empty or
        'df_pontuacoes' not in globals() or df_pontuacoes.empty
    ):
        print("⚠️ Rodadas ou pontuações ainda não disponíveis — classificação não será gerada.")
    
    elif 'classificacao_por_grupo' not in globals():
        print("⚠️ Função 'classificacao_por_grupo' não está definida.")
    
    else:
        # Gerar a classificação da fase 4
        df_resultado, df_resultado_por_jogo = classificacao_por_grupo(
            df_rodadas, df_pontuacoes
        )

        # Salvar cada grupo em uma aba do Excel
        with pd.ExcelWriter("3_classificacao_por_jogo_semi_sula.xlsx") as writer:
            for jogo, df in df_resultado_por_jogo.items():
                df.to_excel(writer, sheet_name=jogo, index=False)

        # Exibir a classificação geral
        df_resultado_jogo_1 = df_resultado[df_resultado["Jogo"] == "Jogo 1 (JG1)"]
        df_resultado_jogo_2 = df_resultado[df_resultado["Jogo"] == "Jogo 2 (JG2)"]

        display(df_resultado_jogo_1, df_resultado_jogo_2)

except Exception as e:
    print(f"❌ Erro ao gerar a classificação: {e}")


,Jogo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
1,Jogo 1 (JG1),Grêmio imortal 37,3,1,0,1,153.30,145.46,7.84,1
0,Jogo 1 (JG1),KING LEONN,3,1,0,1,145.46,153.30,-7.84,2


,Jogo,Nome do Time,Pontos,Vitórias,Empates,Derrotas,Total Cartola,Cartola Sofrido,Saldo Cartola,Posição
2,Jogo 2 (JG2),MauHumor F.C.,6,2,0,0,165.31,126.74,38.57,1
3,Jogo 2 (JG2),lsauer fc,0,0,0,2,126.74,165.31,-38.57,2


{18421230: 'Gig@ntte',
 13951133: 'KING LEONN',
 18344271: 'Laranjja Mecannica',
 24468241: 'Grêmio imortal 37',
 25751748: 'MauHumor F.C.',
 479510: 'TEAM LOPES 99',
 47543456: 'Tabajara de Inhaua FC2',
 44810918: 'lsauer fc'

In [92]:
# Verifica se a variável df_resultado_por_grupo existe e está populada
if 'df_resultado_por_jogo' in locals() and df_resultado_por_jogo:
    # Criar estrutura em formato de dicionário para JSON/JS
    classificacao_quartas_sula = {}

    for jogo, df in df_resultado_por_jogo.items():
        classificacao_quartas_sula[jogo] = []

        # Remove duplicatas
        df = df.drop_duplicates(subset=["Nome do Time"])

        for _, row in df.iterrows():
            classificacao_quartas_sula[jogo].append({
                "posicao": int(row["Posição"]),
                "nome": row["Nome do Time"],
                "pontos": int(row["Pontos"]),
                "vitorias": int(row["Vitórias"]),
                "empates": int(row["Empates"]),
                "derrotas": int(row["Derrotas"]),
                "totalCartola": float(row["Total Cartola"]),
                "cartolaSofrido": float(row["Cartola Sofrido"]),
                "saldoCartola": float(row["Saldo Cartola"])
            })


    # Converter para JSON formatado
    json_str = json.dumps(classificacao_quartas_sula, indent=2, ensure_ascii=False)

    # Salvar como arquivo JS com uma variável global
    with open("3_classificacao_por_jogo_semi_sula.js", "w", encoding="utf-8") as f:
        f.write("const classificacao_semi_sula = ")
        f.write(json_str)
        f.write(";")

    print("✅ Arquivo JS salvo com sucesso.")

else:
    print("⚠️ Classificação das Quartas da Sula indisponíveis. Criando placeholders.")

    # ⚠️ Classificação da fase 4 indisponível — criar estrutura zerada manualmente

    # Dicionário com confrontos definidos
    confrontos_fase_4 = {
        "Jogo 1 (JG1)": ["KING LEONN", "Grêmio imortal 37"],
        "Jogo 2 (JG2)": ["MauHumor F.C.", "lsauer fc"],
        # "Jogo 3 (JG3)": ["MauHumor F.C.", "TEAM LOPES 99"],
        # "Jogo 4 (JG4)": ["Tabajara de Inhaua FC2", "lsauer fc"]
    }

    # Estrutura zerada para o JS
    classificacao_quartas_sula = {}

    for jogo, times in confrontos_fase_4.items():
        classificacao_quartas_sula[jogo] = []
        for nome_time in times:
            classificacao_quartas_sula[jogo].append({
                "posicao": 0,
                "nome": nome_time,
                "pontos": 0,
                "vitorias": 0,
                "empates": 0,
                "derrotas": 0,
                "totalCartola": 0.0,
                "cartolaSofrido": 0.0,
                "saldoCartola": 0.0
            })

    # Salvar como JS
    with open("3_classificacao_por_jogo_semi_sula.js", "w", encoding="utf-8") as f:
        f.write("const classificacao_semi_sula = ")
        json.dump(classificacao_quartas_sula, f, indent=2, ensure_ascii=False)
        f.write(";")

    print("✅ Arquivo JS com classificação zerada gerado com sucesso.")


✅ Arquivo JS salvo com sucesso.


In [93]:
def exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada, jogo=None):
    """
    Exibe os resultados de uma rodada específica, com pontuação e dados dos times.
    """

    if rodada not in df_rodadas["Rodada"].values:
        return pd.DataFrame([{
            "Jogo": jogo or "-",
            "Rodada": rodada,
            "Mandante_Nome": "-",          
            "Mandante_Pontos": "-",
            "Visitante_Nome": "-",           
            "Visitante_Pontos": "-",
        }])

    df_filtrado = df_rodadas[df_rodadas["Rodada"] == rodada]
    if jogo:
        df_filtrado = df_filtrado[df_filtrado["Jogo"] == jogo]

    resultados = []

    for _, row in df_filtrado.iterrows():
        jogo_ = row["Jogo"]
        mandante = row["Mandante_Nome"]
        visitante = row["Visitante_Nome"]

        pontos_mandante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(mandante, None)
        pontos_visitante = df_pontuacoes.get(f"Rodada {rodada}", {}).get(visitante, None)

        resultados.append({
            "Jogo": jogo_,
            "Rodada": rodada,
            "Mandante_Nome": mandante,
            "Mandante_Pontos": pontos_mandante,
            "Visitante_Nome": visitante,
            "Visitante_Pontos": pontos_visitante
        })

    return pd.DataFrame(resultados)


In [94]:
# Exibir resultados da 15ª rodada
df_resultados_rodada_15 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=17)

# Exibir apenas os resultados do Grupo B na 1ª rodada
df_resultados_jogo_1 = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=17, jogo="Jogo 1 (JG1)")

# Exibir
display(df_resultados_rodada_15)

,Jogo,Rodada,Mandante_Nome,Mandante_Pontos,Visitante_Nome,Visitante_Pontos
0,Jogo 1 (JG1),17,KING LEONN,53.09,Grêmio imortal 37,80.10
1,Jogo 2 (JG2),17,MauHumor F.C.,59.81,lsauer fc,44.16


In [95]:
# Criar arquivo com uma aba para cada rodada contendo os resultados detalhados
from pathlib import Path

# Caminho do arquivo de saída
caminho_resultados = "3_resultados_semi_sula.xlsx"

# Descobrir as rodadas únicas no DataFrame
rodadas_disponiveis = sorted(df_rodadas["Rodada"].unique())

with pd.ExcelWriter(caminho_resultados) as writer:
    for rodada in rodadas_disponiveis:
        df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
        nome_aba = f"Rodada {rodada}"
        df_resultados.to_excel(writer, sheet_name=nome_aba, index=False)

print(f"Arquivo salvo com sucesso: {Path(caminho_resultados).resolve()}")

display(df_resultados)

Arquivo salvo com sucesso: C:\Users\ferna\Projetos\GitHub\cartola_fbc\sulamericana\datasets_sula\3_resultados_semi_sula.xlsx


,Jogo,Rodada,Mandante_Nome,Mandante_Pontos,Visitante_Nome,Visitante_Pontos
0,Jogo 1 (JG1),18,Grêmio imortal 37,73.20,KING LEONN,92.37
1,Jogo 2 (JG2),18,lsauer fc,82.58,MauHumor F.C.,105.50


In [96]:
resultados_js = []

for rodada in sorted(df_rodadas["Rodada"].unique()):
    df_resultados = exibir_resultados_rodada(df_rodadas, df_pontuacoes, rodada=rodada)
    
    for _, row in df_resultados.iterrows():
        resultado = {
            "jogo": row["Jogo"],
            "rodada": int(rodada),
            "mandante": {
                "nome": row["Mandante_Nome"],
                "pontos": float(row["Mandante_Pontos"]) if row["Mandante_Pontos"] is not None else None
            },
            "visitante": {
                "nome": row["Visitante_Nome"],
                "pontos": float(row["Visitante_Pontos"]) if row["Visitante_Pontos"] is not None else None               
            },
            "vencedor": (
                "mandante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] > row["Visitante_Pontos"]
                else "visitante" if row["Mandante_Pontos"] is not None and row["Visitante_Pontos"] is not None and row["Mandante_Pontos"] < row["Visitante_Pontos"]
                else "empate" if row["Mandante_Pontos"] == row["Visitante_Pontos"] and row["Mandante_Pontos"] is not None
                else "indefinido"
            )

        }
        resultados_js.append(resultado)


# Exportar para arquivo .js
import json

with open("3_resultados_semi_sula.js", "w", encoding="utf-8") as f:
    f.write("const resultados_semi_sula = ")
    f.write(json.dumps(resultados_js, indent=2, ensure_ascii=False))
    f.write(";")


### Identificando Vencedores das Quartas de Final da Sul Americana

In [97]:
def obter_classificados_e_perdedores(resultados, ids_por_nome):
    classificados = []
    perdedores = []
    jogos_agrupados = {}

    for jogo in resultados:
        chave = jogo['jogo']
        if chave not in jogos_agrupados:
            jogos_agrupados[chave] = []
        jogos_agrupados[chave].append(jogo)

    for chave, partidas in jogos_agrupados.items():
        if len(partidas) < 2:
            continue

        partidas = sorted(partidas, key=lambda x: x['rodada'])
        ida, volta = partidas

        if ida['mandante']['pontos'] is None or volta['mandante']['pontos'] is None:
            continue

        time1 = ida['mandante']['nome']
        time2 = ida['visitante']['nome']

        pontos_time1 = ida['mandante']['pontos'] + volta['visitante']['pontos']
        pontos_time2 = ida['visitante']['pontos'] + volta['mandante']['pontos']

        if pontos_time1 > pontos_time2:
            vencedor = time1
            perdedor = time2
        elif pontos_time2 > pontos_time1:
            vencedor = time2
            perdedor = time1
        else:
            # Se empatar, decidir por critério de desempate se desejar
            vencedor = "EMPATE"
            perdedor = "EMPATE"

        classificados.append({
            "jogo": chave,
            "classificado_nome": vencedor,
            "classificado_id": ids_por_nome.get(vencedor) if vencedor in ids_por_nome else None
        })

        perdedores.append({
            "jogo": chave,
            "perdedor_nome": perdedor,
            "perdedor_id": ids_por_nome.get(perdedor) if perdedor in ids_por_nome else None
        })

    return classificados, perdedores


In [99]:
# Inverter os nomes
ids_por_nome = {v: k for k, v in nomes_por_id.items()}

# Ler o resultados_fase_4.js
with open("3_resultados_semi_sula.js", "r", encoding="utf-8") as f:
    conteudo = f.read()

conteudo_json = conteudo.replace("const resultados_semi_sula = ", "").strip().rstrip(";")
resultados = json.loads(conteudo_json)

# Obter classificados e perdedores
classificados, perdedores = obter_classificados_e_perdedores(resultados, ids_por_nome)

# Salvar os classificados
df_classificados = pd.DataFrame(classificados)
df_classificados.to_excel("classificados_fase_3.xlsx", index=False)

with open("classificados_fase_3.js", "w", encoding="utf-8") as f:
    f.write("const classificadosFase3 = ")
    json.dump(classificados, f, ensure_ascii=False, indent=2)
    f.write(";")

# Salvar os perdedores
df_perdedores = pd.DataFrame(perdedores)
df_perdedores.to_excel("perdedores_fase_3.xlsx", index=False)

with open("perdedores_fase_3.js", "w", encoding="utf-8") as f:
    f.write("const perdedoresFase3 = ")
    json.dump(perdedores, f, ensure_ascii=False, indent=2)
    f.write(";")

print("✅ Classificados e perdedores salvos com sucesso em arquivos .xlsx e .js.")

# Exibir para validar
print("Classificados encontrados:", classificados)
print("Perdedores encontrados:", perdedores)


✅ Classificados e perdedores salvos com sucesso em arquivos .xlsx e .js.
Classificados encontrados: [{'jogo': 'Jogo 1 (JG1)', 'classificado_nome': 'Grêmio imortal 37', 'classificado_id': 24468241}, {'jogo': 'Jogo 2 (JG2)', 'classificado_nome': 'MauHumor F.C.', 'classificado_id': 25751748}]
Perdedores encontrados: [{'jogo': 'Jogo 1 (JG1)', 'perdedor_nome': 'KING LEONN', 'perdedor_id': 13951133}, {'jogo': 'Jogo 2 (JG2)', 'perdedor_nome': 'lsauer fc', 'perdedor_id': 44810918}]


In [100]:
# def obter_classificados_com_id(resultados, ids_por_nome):
#     classificados = []
#     jogos_agrupados = {}
#     for jogo in resultados:
#         chave = jogo['jogo']
#         if chave not in jogos_agrupados:
#             jogos_agrupados[chave] = []
#         jogos_agrupados[chave].append(jogo)

#     for chave, partidas in jogos_agrupados.items():
#         if len(partidas) < 2:
#             continue

#         partidas = sorted(partidas, key=lambda x: x['rodada'])
#         ida, volta = partidas

#         if ida['mandante']['pontos'] is None or volta['mandante']['pontos'] is None:
#             continue

#         time1 = ida['mandante']['nome']
#         time2 = ida['visitante']['nome']

#         pontos_time1 = ida['mandante']['pontos'] + volta['visitante']['pontos']
#         pontos_time2 = ida['visitante']['pontos'] + volta['mandante']['pontos']

#         if pontos_time1 > pontos_time2:
#             vencedor = time1
#         elif pontos_time2 > pontos_time1:
#             vencedor = time2
#         else:
#             vencedor = "EMPATE"

#         classificados.append({
#             "jogo": chave,
#             "classificado_nome": vencedor,
#             "classificado_id": ids_por_nome.get(vencedor) if vencedor in ids_por_nome else None
#         })

#     return classificados


In [101]:
# # Inverter os nomes
# ids_por_nome = {v: k for k, v in nomes_por_id.items()}

# # Ler o resultados_quartas_sula.js (como já fizemos antes)
# with open("3_resultados_semi_sula.js", "r", encoding="utf-8") as f:
#     conteudo = f.read()

# conteudo_json = conteudo.replace("const resultados_semi_sula = ", "").strip().rstrip(";")
# resultados = json.loads(conteudo_json)

# # Obter os classificados
# classificados = obter_classificados_com_id(resultados, ids_por_nome)

# # Salvar
# df_classificados = pd.DataFrame(classificados)
# df_classificados.to_excel("3_classificados_semi_sula.xlsx", index=False)

# with open("3_classificados_semi_sula.js", "w", encoding="utf-8") as f:
#     f.write("const classificados_semi_sula = ")
#     json.dump(classificados, f, ensure_ascii=False, indent=2)
#     f.write(";")
# print("✅ Classificados salvos com sucesso em '3_classificados_semi_sula.xlsx' e '3_classificados_semi_sula.js'.")

# # Exibir para validar
# print("Classificados encontrados:", classificados)
